## Evaluate Leave One Image Out analysis

In [1]:
import pathlib
import numpy as np
import pandas as pd

from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_curve, auc

In [2]:
def compute_avg_rank_and_pvalue(grouped_df):
    ranks = []
    p_values = []

    for _, group in grouped_df.groupby("Cell_UUID"):
        # Sort predicted probabilities in descending order
        sorted_probs = group.sort_values(by="Predicted_Probability", ascending=False).reset_index(drop=True)
        
        # Get the rank of the true class
        rank = sorted_probs[sorted_probs["Mitocheck_Phenotypic_Class"] == sorted_probs["Model_Phenotypic_Class"]].index[0] + 1
        
        # Get the p-value (predicted probability) of the true class
        p_value = sorted_probs.loc[rank - 1, "Predicted_Probability"]
        
        ranks.append(rank)
        p_values.append(p_value)

    # Calculate average rank and p-value for the group
    avg_rank = sum(ranks) / len(ranks)
    avg_p_value = sum(p_values) / len(p_values)
    
    # Calculate IQR and min/max within IQR for ranks
    iqr_rank = np.percentile(ranks, 75) - np.percentile(ranks, 25)
    min_iqr_rank = np.percentile(ranks, 25)
    max_iqr_rank = np.percentile(ranks, 75)
    
    # Calculate IQR and min/max within IQR for p-values
    iqr_p_value = np.percentile(p_values, 75) - np.percentile(p_values, 25)
    min_iqr_p_value = np.percentile(p_values, 25)
    max_iqr_p_value = np.percentile(p_values, 75)
    
    # Count number of comparisons
    count = len(ranks)
    
    return avg_rank, avg_p_value, min_iqr_rank, max_iqr_rank, min_iqr_p_value, max_iqr_p_value, count

In [3]:
# Set I/O
proba_dir = pathlib.Path("evaluations", "LOIO_probas")
loio_file = pathlib.Path(proba_dir, "compiled_LOIO_probabilites_withshuffled.tsv")

output_summary_file = pathlib.Path(proba_dir, "LOIO_summary_ranks_withshuffled.tsv")
output_summary_phenotype_file = pathlib.Path(proba_dir, "LOIO_summary_ranks_perphenotype_withshuffled.tsv")

In [4]:
loio_df = pd.read_csv(loio_file, sep="\t", index_col=0)

print(loio_df.shape)
loio_df.head()

(257580, 9)


,Cell_UUID,Metadata_DNA,Mitocheck_Phenotypic_Class,Model_Feature_Type,Model_C,Model_l1_ratio,Model_type,Model_Phenotypic_Class,Predicted_Probability
0,0008551d-e7f6-4351-b680-140c3661cb59,LT0109_38/LT0109_38_381_87.tif,Interphase,CP,10.0,0.6,final,ADCCM,0.000006
1,0008551d-e7f6-4351-b680-140c3661cb59,LT0109_38/LT0109_38_381_87.tif,Interphase,CP,0.1,0.8,shuffled,ADCCM,0.035385
2,0008551d-e7f6-4351-b680-140c3661cb59,LT0109_38/LT0109_38_381_87.tif,Interphase,CP,10.0,0.6,final,Anaphase,0.162337
3,0008551d-e7f6-4351-b680-140c3661cb59,LT0109_38/LT0109_38_381_87.tif,Interphase,CP,0.1,0.8,shuffled,Anaphase,0.003718
4,0008551d-e7f6-4351-b680-140c3661cb59,LT0109_38/LT0109_38_381_87.tif,Interphase,CP,10.0,0.6,final,Apoptosis,0.807639


In [5]:
phenotype_classes = loio_df.Mitocheck_Phenotypic_Class.unique().tolist()
phenotype_classes

['Interphase',
 'MetaphaseAlignment',
 'OutOfFocus',
 'Elongated',
 'Apoptosis',
 'Prometaphase',
 'Large',
 'Polylobed',
 'SmallIrregular',
 'Binuclear',
 'Hole',
 'ADCCM',
 'Metaphase',
 'Anaphase',
 'Grape']

## Get average ranks and p value of correct prediction

- Per Image
- Per model type (final vs. shuffled)
- Phenotype
- Feature Space

In [6]:
# Calculate average rank for each Metadata_DNA
avg_ranks = (
    loio_df.groupby(["Metadata_DNA", "Model_type", "Mitocheck_Phenotypic_Class", "Model_Feature_Type"])
    .apply(compute_avg_rank_and_pvalue)
    .reset_index()
)

avg_ranks.columns = ["Metadata_DNA", "Model_type", "Mitocheck_Phenotypic_Class", "Model_Feature_Type", "Average_Scores"]

loio_scores_df = (
    pd.concat([
        avg_ranks.drop(columns="Average_Scores"),
        pd.DataFrame(avg_ranks.Average_Scores.tolist(), columns=[
            "Average_Rank", "Average_P_Value", "Min_IQR_Rank", "Max_IQR_Rank", 
            "Min_IQR_P_Value", "Max_IQR_P_Value",  "Count"
        ])
    ], axis="columns")
)

loio_scores_df.to_csv(output_summary_file, index=False, sep="\t")

print(loio_scores_df.shape)
loio_scores_df.head()

(3954, 11)


,Metadata_DNA,Model_type,Mitocheck_Phenotypic_Class,Model_Feature_Type,Average_Rank,Average_P_Value,Min_IQR_Rank,Max_IQR_Rank,Min_IQR_P_Value,Max_IQR_P_Value,Count
0,LT0003_40/LT0003_40_149_83.tif,final,Polylobed,CP,2.333333,0.652988,1.0,3.0,0.484308,0.979077,3
1,LT0003_40/LT0003_40_149_83.tif,final,Polylobed,CP_and_DP,1.666667,0.630175,1.0,2.0,0.461726,0.944200,3
2,LT0003_40/LT0003_40_149_83.tif,final,Polylobed,DP,1.333333,0.701701,1.0,1.5,0.581234,0.934144,3
3,LT0003_40/LT0003_40_149_83.tif,shuffled,Polylobed,CP,4.666667,0.118837,2.5,6.0,0.097702,0.155380,3
4,LT0003_40/LT0003_40_149_83.tif,shuffled,Polylobed,CP_and_DP,8.000000,0.203849,5.0,11.5,0.000581,0.305773,3


## Get average ranks and p value of correct prediction

- Per model type (final vs. shuffled)
- Per Phenotype
- Per Feature space

(i.e., not on a per-image basis)

In [7]:
# Calculate average rank for each Metadata_DNA
avg_ranks = (
    loio_df.groupby(["Model_type", "Mitocheck_Phenotypic_Class", "Model_Feature_Type"])
    .apply(compute_avg_rank_and_pvalue)
    .reset_index()
)

avg_ranks.columns = ["Model_type", "Mitocheck_Phenotypic_Class", "Model_Feature_Type", "Average_Scores"]

loio_scores_df = (
    pd.concat([
        avg_ranks.drop(columns="Average_Scores"),
        pd.DataFrame(avg_ranks.Average_Scores.tolist(), columns=[
            "Average_Rank", "Average_P_Value", "Min_IQR_Rank", "Max_IQR_Rank", 
            "Min_IQR_P_Value", "Max_IQR_P_Value",  "Count"
        ])
    ], axis="columns")
)

loio_scores_df.to_csv(output_summary_phenotype_file, index=False, sep="\t")

print(loio_scores_df.shape)
loio_scores_df.head()

(90, 10)


,Model_type,Mitocheck_Phenotypic_Class,Model_Feature_Type,Average_Rank,Average_P_Value,Min_IQR_Rank,Max_IQR_Rank,Min_IQR_P_Value,Max_IQR_P_Value,Count
0,final,ADCCM,CP,2.273684,0.404648,1.0,3.0,0.042986,0.678284,95
1,final,ADCCM,CP_and_DP,2.294737,0.470747,1.0,3.0,0.090173,0.793014,95
2,final,ADCCM,DP,2.736842,0.415654,1.0,3.0,0.084356,0.698022,95
3,final,Anaphase,CP,6.702381,0.264648,1.0,12.0,0.000184,0.643928,84
4,final,Anaphase,CP_and_DP,6.761905,0.271571,1.0,13.0,0.000935,0.676832,84
